In [1]:
# icdar17 dataset 중에서 "Latin & codec에 정의된 문자"인 데이터만 추출

In [2]:
import os
import shutil
import glob
import re

In [3]:
datasets_path = '/opt/ml/project/models/datasets'

In [4]:
file_path = datasets_path + '/train_gts_icdar17'
file_names = sorted(os.listdir(file_path))
image_path = datasets_path + '/train_images_icdar17'

In [5]:
new_file_path = file_path + '_alphabet'
new_image_path = image_path + '_alphabet'
if not os.path.isdir(new_file_path):
    os.makedirs(new_file_path)
if not os.path.isdir(new_image_path):
    os.makedirs(new_image_path)

In [6]:
with open('/opt/ml/project/models/modules/utils/codec.txt', 'r') as f:
    keys = f.readlines()[0]
    print(keys)
regular_char = re.compile(r"[ 0-9a-zA-Z@!#$%&'()+,-.·/:;=?´_~*°<>\"\'\[\]]")

  0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ@!#$%&'()+,-.·/:;=?´_~*°<>"'[]\


In [7]:
for file_name in file_names:
    src = os.path.join(file_path, file_name)
    try:
        with open(src, 'r', encoding='utf-8') as f:
            new_texts = ''
            lines = f.readlines()
            eng_exist = False
            for line in lines:
                texts = line[:-1].split(',')
                bbox = ','.join(texts[:8])
                script = texts[8]
                trans = ','.join(texts[9:])
                
                possible_word = False
                if script == 'Latin':
                    possible_word = True
                    for char in trans:
                        if not regular_char.match(char):
                            possible_word = False
                
                if trans == '':
                    continue
                elif possible_word:
                    eng_exist = True
                    new_texts += bbox+','+trans+'\n'
                else:
                    new_texts += bbox+',###\n'
    except:
        print(file_name)
        continue
    if eng_exist:
        new_src = os.path.join(new_file_path, file_name)
        with open(new_src,'w') as f:
            f.write(new_texts)
        image_name = 'img_' + file_name.split('_')[2].split('.')[0]+'.*'
        image_name = glob.glob(image_path+'/'+image_name)[0].split('/')[-1]
        image_src = os.path.join(image_path, image_name)
        new_image_src = os.path.join(new_image_path, image_name)
        shutil.copyfile(image_src, new_image_src)
        # print(image_src, new_image_src)

.ipynb_checkpoints
